<a href="https://colab.research.google.com/github/peeyushsinghal/image-segmentation/blob/main/roads_serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Serving Image segmentation models using Flask on Google Colab

In [ ]:
!pip install flask-ngrok

In [ ]:
from google.colab import drive
# drive.flush_and_unmount()
# drive.mount('/content/gdrive',force_remount=True)

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Constants
base_dir = '/content/gdrive/MyDrive/image-segment-dataset/massachusetts-roads-dataset/road_segmentation_ideal' # gdrive


In [ ]:
import os
import cv2

flask_app = os.path.join(base_dir,'flask_app') # flask app directory 
upload_folder = os.path.join(flask_app,'upload_folder') # upload folder within flask app dir
templates = os.path.join(flask_app,'templates') # folder to store. templates

dirs = [flask_app, upload_folder,templates]

for dir in dirs:
  try:
      if not os.path.exists(dir):
        os.makedirs(dir)
        print(f"{dir} directory is created")
      else:
        print(f"{dir} directory is already present")
  except:
      print(f"Problem encountered while creating {dir} directory")


/content/gdrive/MyDrive/image-segment-dataset/massachusetts-roads-dataset/road_segmentation_ideal/flask_app directory is already present
/content/gdrive/MyDrive/image-segment-dataset/massachusetts-roads-dataset/road_segmentation_ideal/flask_app/upload_folder directory is already present
/content/gdrive/MyDrive/image-segment-dataset/massachusetts-roads-dataset/road_segmentation_ideal/flask_app/templates directory is already present


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,flash,request,redirect,url_for,render_template
import urllib.request
import os
from werkzeug.utils import secure_filename
import cv2
import tensorflow as tf
# from PIL import Image
import matplotlib.pyplot as plt # for plotting
from matplotlib import cm # for storing predicted image
import math # to execute math operations for image processing
import numpy as np


# import tf.keras.models import load_model



ALLOWED_EXTENSIONS = set(['png'])
UPLOAD_FOLDER = upload_folder
app = Flask(__name__,template_folder=templates,static_folder=upload_folder)

app.secret_key ='secret key'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16*1500*1500

run_with_ngrok(app)  

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def load_model(model_selected = 'unet_pretrained_mobilenetv2'):
  model_dir = os.path.join(base_dir,'models') # models directory 
  model_save_dir = os.path.join(model_dir,str(model_selected)) # directory for the selected model
  model_final_save_dir = os.path.join(model_save_dir,'model_save') # final save directory for the selected model

  try:
    if os.path.exists(model_dir) or os.path.exists(model_save_dir) or os.path.exists(model_final_save_dir):
      print("Model Directories are accessible")
  except:
    print("Model Directories are not present / accessible")

  custom_objects ={'dice_loss':tf.keras.losses,'metric_iou':tf.keras.metrics, 'dice_coef': tf.keras.metrics}
  loaded_model = tf.keras.models.load_model(model_final_save_dir,custom_objects=custom_objects)
  # # loaded_model = tf.keras.models.load_model(model_final_save_dir,custom_objects=custom_objects,compile=False) # compile = False, does not make model eligible for further fit and evaluate

  return loaded_model

def predict_loaded_model(x, loaded_model,
                         model_img_size = 512,original_img_size=1500, 
                         threshold = 0.5
                         ):
  
  # x = read_input_image(test_x)

  split = math.ceil(original_img_size / model_img_size) # Number of split (tile size in one dimension)
  
  exact_split = True
  if original_img_size % model_img_size != 0:
    exact_split = False

  split_size = int(original_img_size / split) # tile size or split size
  assert (original_img_size % split) == 0, "Please change the model image size, accepted model image sizes are 128,256,512,1024"

  tiles = [x[w:w+split_size,h:h+split_size] for w in range(0,original_img_size,split_size) for h in range(0,original_img_size,split_size)]
  # print (f" The image is split into {len(tiles)} parts")

  rows = int(math.sqrt(len(tiles)))
  columns = int(math.sqrt(len(tiles)))
  
  tile_counter = 0
  image_row = []
  for row in range(0,rows):
    list_col = []
    for col in range(0,columns):
      img = tiles[tile_counter]
      tile_counter += 1
      img = cv2.resize(img, (model_img_size, model_img_size),
                       interpolation= cv2.INTER_LINEAR) # increase in size to model size
      ''' model.predict goes here'''
      img = loaded_model.predict(np.expand_dims(img, axis=0))[0] > threshold
      # print("shape after predict",img.shape)
      img = cv2.resize(np.float32(img), (split_size, split_size),
                       interpolation= cv2.INTER_AREA) # shinking in size to image split size
      # print("shape after reduction in size",img.shape)
      list_col.append(img)           
    all_row_imgs = np.concatenate(list_col, axis=1)
    image_row.append(all_row_imgs)
  y_pred = np.concatenate(image_row, axis=0)


  return y_pred

def read_input_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    # x = cv2.resize(x, (ORIGINAL_IMG_SIZE, ORIGINAL_IMG_SIZE))
    x = x/255.0
    return x

@app.route("/")
# def home():
#     return f"Massachusetts-road-dataset Image Segmentation "
def upload_form():
 	return render_template('upload.html')

@app.route('/', methods=['POST'])
def upload_image():
	if 'file' not in request.files:
		flash('No file part')
		return redirect(request.url)
	file = request.files['file']
	if file.filename == '':
		flash('No image selected for uploading')
		return redirect(request.url)
	if file and allowed_file(file.filename):
		filename = secure_filename(file.filename)
		file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
		#print('upload_image filename: ' + filename)
		flash('Image successfully uploaded and displayed below')
		return render_template('upload.html', filename=filename)
	else:
		flash('Allowed image type is png only')
		return redirect(request.url)

  
@app.route('/display/<filename>')
def display_image(filename):

  # print('display_image filename: ' + filename)
  return redirect(url_for('static', filename= filename), code=301)

@app.route('/predict/<filename>')
def predicted_image(filename):
  img = read_input_image(os.path.join(app.config['UPLOAD_FOLDER'],filename))
  loaded_model = load_model(model_selected = 'unet_pretrained_mobilenetv2')
  y_pred = predict_loaded_model(img, loaded_model,
                         model_img_size = 512,original_img_size=1500, 
                         threshold = 0.5)
s  fname_updated = 'mask_'+str(filename.split("/")[-1])
  fname = os.path.join(app.config['UPLOAD_FOLDER'],fname_updated)
  plt.imsave(fname, y_pred, cmap=cm.gray,format ='png')
  return redirect(url_for('static', filename= fname_updated), code=301)


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://de70b884fa4f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Jul/2021 21:06:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 21:06:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Jul/2021 21:06:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 21:06:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 21:06:46] "GET /display/img-1.png HTTP/1.1" 301 -
127.0.0.1 - - [11/Jul/2021 21:06:46] "GET /upload_folder/img-1.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2021 21:07:08] "POST / HTTP/1.1" 200 -


(1500, 1500, 3)
Model Directories are accessible
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1500, 1500)


127.0.0.1 - - [11/Jul/2021 21:07:54] "GET /predict/img-1.png HTTP/1.1" 301 -
127.0.0.1 - - [11/Jul/2021 21:07:54] "GET /upload_folder/mask_img-1.png HTTP/1.1" 200 -
